# Building content-based filtering

### Creating several categories


In [1]:
import pandas as pd
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype
import matplotlib.pyplot  as plt
import numpy as np
import feather

from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial.distance import pdist,squareform 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA

from scipy.sparse import csr_matrix




from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.options.display.max_rows = 500
pd.options.display.max_columns = 200 
pd.set_option('display.float_format', lambda x: '%.3f' % x) #display numbers as decimals instead of scientific value

data_filepath = 'movie_metadata_cleaned_v1.feather'


In [2]:
df = feather.read_dataframe(data_filepath)
movies_count = df.shape[0]
print (   movies_count ,'movies imported')

df_movies = df.copy()
df_movies['movie_id'] = df_movies.index
df_movies = df_movies.set_index(df['movie_title'])
movies_list = list(df['movie_title'])

4916 movies imported


In [3]:
df.describe(include='all')

,movie_title,title_year,imdb_score,country,language,genres,plot_keywords,director_name,director_facebook_likes,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,cast_total_facebook_likes,movie_facebook_likes,num_user_for_reviews,num_critic_for_reviews,num_voted_users,gross,budget,content_rating,color,duration,aspect_ratio
count,4916,4916.000,4916.000,4916,4916,4916,4916,4916,4916.000,4916,4916,4916,4916.000,4916.000,4916.000,4916.000,4916.000,4916.000,4916.000,4916.000,4916.000,4916.000,4916,4916,4916.000,4916.000
unique,4916,nan,nan,66,48,914,4757,2398,nan,2096,3031,3520,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,18,2,nan,nan
top,Faithful,nan,nan,USA,English,Drama,,,nan,Robert De Niro,Morgan Freeman,,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,R,Color,nan,nan
freq,1,nan,nan,3710,4582,232,152,102,nan,48,18,23,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,2367,4712,nan,nan
mean,NaN,2002.453,6.437,NaN,NaN,NaN,NaN,NaN,676.677,NaN,NaN,NaN,6485.200,1617.624,628.313,9579.733,7348.296,266.525,136.617,82645.242,47779121.547,33576517.195,NaN,NaN,107.092,2.096
std,NaN,12.315,1.128,NaN,NaN,NaN,NaN,NaN,2805.134,NaN,NaN,NaN,15098.229,4006.861,1622.642,18164.359,19206.016,372.547,120.418,138322.057,61184608.962,38653472.075,NaN,NaN,25.247,0.289
min,NaN,1916.000,1.600,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,NaN,0.000,0.000,0.000,0.000,0.000,0.000,0.000,5.000,162.000,218.000,NaN,NaN,7.000,1.180
25%,NaN,1999.000,5.800,NaN,NaN,NaN,NaN,NaN,6.000,NaN,NaN,NaN,606.000,275.000,129.000,1394.750,0.000,62.000,47.000,8361.750,8052632.500,7000000.000,NaN,NaN,93.000,1.850
50%,NaN,2005.000,6.600,NaN,NaN,NaN,NaN,NaN,44.000,NaN,NaN,NaN,981.500,592.000,362.000,3049.000,159.000,152.000,106.500,33132.500,36948242.000,23000000.000,NaN,NaN,103.000,2.220
75%,NaN,2010.000,7.200,NaN,NaN,NaN,NaN,NaN,188.000,NaN,NaN,NaN,11000.000,912.000,631.000,13616.750,2000.000,319.250,191.000,93772.750,50875351.250,40000000.000,NaN,NaN,118.000,2.350


In [4]:
def replace_spaces(feature):
    feature = feature.str.replace(' ', '_')
    feature = feature.str.replace('.', '')
    feature = feature.str.replace('|', ' ')
    return feature

df_2 = df.copy()
df_2 = df_2.set_index(df_2['movie_title'])
object_only = df_2.select_dtypes(include=['object'])
df_2[object_only.columns] = object_only.apply(replace_spaces, axis=1)


In [5]:
df_2.head()

,movie_title,title_year,imdb_score,country,language,genres,plot_keywords,director_name,director_facebook_likes,actor_1_name,actor_2_name,actor_3_name,actor_1_facebook_likes,actor_2_facebook_likes,actor_3_facebook_likes,cast_total_facebook_likes,movie_facebook_likes,num_user_for_reviews,num_critic_for_reviews,num_voted_users,gross,budget,content_rating,color,duration,aspect_ratio
movie_title,,,,,,,,,,,,,,,,,,,,,,,,,,
Gods and Monsters,Gods_and_Monsters,1998.000,7.500,USA,English,Biography Drama,friendship gardener gay_interest homosexual me...,Bill_Condon,386.000,Brendan_Fraser,Lynn_Redgrave,Kevin_J_O'Connor,3000.000,258.000,248.000,4238,0,229.000,127.000,24977,6390032.000,3500000.000,R,_Black_and_White,105.000,2.350
Runner Runner,Runner_Runner,2013.000,5.600,USA,English,Crime Thriller,costa_rica fbi_agent gambling money online_gam...,Brad_Furman,65.000,Justin_Timberlake,John_Heard,David_Costabile,3000.000,697.000,681.000,5706,0,95.000,184.000,52069,19316646.000,30000000.000,R,Color,88.000,2.350
Antitrust,Antitrust,2001.000,6.100,USA,English,Action Crime Drama Thriller,boss communications computer genius job,Peter_Howitt,29.000,Tyler_Labine,Richard_Roundtree,Ned_Bellamy,779.000,240.000,91.000,1297,0,221.000,123.000,25558,10965209.000,30000000.000,PG-13,Color,109.000,2.350
Lady in White,Lady_in_White,1988.000,6.700,USA,English,Fantasy Horror Mystery Thriller,1960s boy ghost murder small_town,Frank_LaLoggia,7.000,Alex_Rocco,Lukas_Haas,Katherine_Helmond,968.000,733.000,339.000,2545,0,69.000,130.000,4695,1705139.000,4700000.000,R,Color,118.000,1.850
Dead Man Down,Dead_Man_Down,2013.000,6.500,USA,English,Action Crime Drama Thriller,crime_lord dead_body dead_body_in_a_freezer ne...,Niels_Arden_Oplev,76.000,Dominic_Cooper,Isabelle_Huppert,James_Biberi,3000.000,678.000,174.000,4253,0,127.000,217.000,54147,10880926.000,30000000.000,R,Color,118.000,2.350


In [6]:
def calculate_dist(df_, min_words_freq = 1):
    df = df_.copy()
    ######### Words columns ############
    df_words_cols = df.select_dtypes(include=['object']).copy()
    df.loc[:,'words']=''
    for column in df_words_cols.columns:
        df.loc[:,'words'] +=df.loc[:,column]+' ' #.T.head(1).T+' '
        if column in ['actor_1_name', 'director_name', 'genres']:
            df.loc[:,'words'] +=df.loc[:,column]+' '
        
    if df_words_cols.shape[1]>0 :
        vectorizer  = TfidfVectorizer(min_df = min_words_freq+1)
        words_freq_csrm = vectorizer.fit_transform(list(df['words']))
        #df_freq = pd.DataFrame(words_freq_csrm.toarray(),index=movies_list, columns=vectorizer.get_feature_names())
        #words_frequencies = pd.Series(df_freq.astype(bool).sum(axis=0))
        #keep_words_i = list(np.where(words_frequencies>min_words_freq))[0]
        #words_freq_csrm= words_freq_csrm[:,keep_words_i]
        words_features = words_freq_csrm.toarray()

    ###### Numeric columns #########
    df_numeric_cols = df.select_dtypes(include=['float64']).copy()
    
    if df_numeric_cols.shape[1]>0 :
        scaler = MinMaxScaler((0.5,2))
        numeric_features = scaler.fit_transform(df_numeric_cols)
        if df_words_cols.shape[1]>0 :
            all_features = np.hstack((words_features, numeric_features))
            n_components = int(round(words_features.shape[1]/3, 0))+numeric_features.shape[1]
        else:
            all_features = numeric_features
            n_components = numeric_features.shape[1]
    else:
        all_features = words_features
        n_components = int(round(words_features.shape[1]/3, 0))
        
    
    print('Reducing to',n_components,'principal components out of',all_features.shape[1],'features')
    ## => indiquer la variance
    pca = PCA(n_components=n_components) # We keep 1/3
    pca_features = pca.fit_transform(all_features)
    ####### Dist matrix ############
    pdist_matrix = pdist(pca_features)
    pdist_square = squareform(pdist_matrix)
    df_pdist = pd.DataFrame(pdist_square, index = movies_list, columns =movies_list )
    
    return df_pdist



In [7]:

def display_closest(movie_name, columns_names, df_pdist, n=5, min_words_freq = 1, recalc=False):
    if (recalc or df_pdist.shape[0]==0):
        df_pdist = calculate_dist(df_2[columns_names], min_words_freq=min_words_freq)
    df_display = df_2[columns_names].copy()
    df_display['distance'] = df_pdist[movie_name].copy()
    display(df_display.loc[df_display['distance'].sort_values().iloc[1:n+1].index, :])
    return df_pdist
    
df_pdist_overall = pd.DataFrame()
df_pdist_genres = pd.DataFrame()
df_pdist_cast = pd.DataFrame()
df_pdist_success= pd.DataFrame()
df_pdist_pict= pd.DataFrame()
def display_categories(movie_name, n=5):
    global df_pdist_overall
    global df_pdist_genres
    global df_pdist_cast
    global df_pdist_success
    global df_pdist_pict

    print('Similar overall')
    df_pdist_overall = display_closest(movie_name, n=n, min_words_freq=2
                                      , df_pdist = df_pdist_overall
                                      , columns_names = ['genres', 'director_name', 'actor_1_name','title_year', 'country', 'language', 'imdb_score', 'content_rating', 'color'])
    
    print('Similar genre')
    df_pdist_genres = display_closest(movie_name, n=n, min_words_freq=5
                                      , df_pdist = df_pdist_genres
                                      , columns_names = ['genres', 'plot_keywords'])
    print('Similar picture')
    df_pdist_pict = display_closest(movie_name, n=n
                                      , df_pdist = df_pdist_pict
                                      , columns_names = ['director_name', 'color', 'title_year', 'aspect_ratio', 'duration', 'country', 'language'])
    
    print('Similar cast')
    df_pdist_cast = display_closest(movie_name, n=n
                                      , df_pdist = df_pdist_cast
                                      , columns_names = ['actor_1_name', 'actor_2_name', 'actor_3_name', 'title_year'])
    print('Similar success')
    df_pdist_success = display_closest(movie_name, n=n
                                      , df_pdist = df_pdist_success
                                      , columns_names = ['imdb_score', 'imdb_score', 'gross', 'num_user_for_reviews', 'movie_facebook_likes',  'num_critic_for_reviews', 'num_voted_users'])
  


In [8]:
df.columns

Index(['movie_title', 'title_year', 'imdb_score', 'country', 'language',
       'genres', 'plot_keywords', 'director_name', 'director_facebook_likes',
       'actor_1_name', 'actor_2_name', 'actor_3_name',
       'actor_1_facebook_likes', 'actor_2_facebook_likes',
       'actor_3_facebook_likes', 'cast_total_facebook_likes',
       'movie_facebook_likes', 'num_user_for_reviews',
       'num_critic_for_reviews', 'num_voted_users', 'gross', 'budget',
       'content_rating', 'color', 'duration', 'aspect_ratio'],
      dtype='object')

In [9]:
display_categories('Fight Club')

Similar overall
Reducing to 354 principal components out of 1058 features


,genres,director_name,actor_1_name,title_year,country,language,imdb_score,content_rating,color,distance
movie_title,,,,,,,,,,
The Curious Case of Benjamin Button,Drama Fantasy Romance,David_Fincher,Brad_Pitt,2008.000,USA,English,7.800,PG-13,Color,0.466
The Social Network,Biography Drama,David_Fincher,Andrew_Garfield,2010.000,USA,English,7.700,PG-13,Color,0.826
The Game,Drama Mystery Thriller,David_Fincher,Deborah_Kara_Unger,1997.000,USA,English,7.800,R,Color,0.832
Gone Girl,Crime Drama Mystery Thriller,David_Fincher,Patrick_Fugit,2014.000,USA,English,8.100,R,Color,0.833
The Tree of Life,Drama Fantasy,Terrence_Malick,Brad_Pitt,2011.000,USA,English,6.700,PG-13,Color,0.899


Similar genre
Reducing to 271 principal components out of 813 features


,genres,plot_keywords,distance
movie_title,,,
The Emperor's Club,Drama,classroom fund_raising headmaster reunion roman,0.452
Lost in Translation,Drama,fish_out_of_water japanese loneliness older_ma...,0.467
Requiem for a Dream,Drama,addiction diet_pill drug_addiction fast_motion...,0.474
Frost/Nixon,Drama,american_president political_drama scandal us_...,0.479
The Master,Drama,drifter past_life_regression psychological_tes...,0.497


Similar picture
Reducing to 322 principal components out of 960 features


,director_name,color,title_year,aspect_ratio,duration,country,language,distance
movie_title,,,,,,,,
The Game,David_Fincher,Color,1997.000,2.350,129.000,USA,English,0.071
Se7en,David_Fincher,Color,1995.000,2.350,127.000,USA,English,0.092
Alien 3,David_Fincher,Color,1992.000,2.350,145.000,USA,English,0.106
Panic Room,David_Fincher,Color,2002.000,2.350,112.000,USA,English,0.122
Zodiac,David_Fincher,Color,2007.000,2.350,162.000,USA,English,0.124


Similar cast
Reducing to 790 principal components out of 2367 features


,actor_1_name,actor_2_name,actor_3_name,title_year,distance
movie_title,,,,,
Sinbad: Legend of the Seven Seas,Brad_Pitt,Adriano_Giannini,Timothy_West,2003.000,0.537
Babel,Brad_Pitt,Harriet_Walter,Dermot_Crowley,2006.000,0.544
Johnny Suede,Brad_Pitt,Tina_Louise,Nick_Cave,1991.000,0.546
The Tree of Life,Brad_Pitt,Tye_Sheridan,Fiona_Shaw,2011.000,0.570
Ocean's Twelve,Brad_Pitt,Julia_Roberts,Mini_Anden,2004.000,0.657


Similar success
Reducing to 5 principal components out of 5 features


,imdb_score,imdb_score,gross,num_user_for_reviews,movie_facebook_likes,num_critic_for_reviews,num_voted_users,distance
movie_title,,,,,,,,
Memento,8.500,8.500,25530884.000,2067.000,40000,274.000,845580,0.290
Pulp Fiction,8.900,8.900,107930000.000,2195.000,45000,215.000,1324680,0.327
Donnie Darko,8.100,8.100,727883.000,2110.000,33000,283.000,580999,0.330
Kill Bill: Vol. 1,8.100,8.100,70098138.000,2105.000,13000,354.000,735784,0.332
The Matrix,8.700,8.700,171383253.000,3646.000,25000,313.000,1217752,0.334


In [10]:
display_categories('Django Unchained')

Similar overall


,genres,director_name,actor_1_name,title_year,country,language,imdb_score,content_rating,color,distance
movie_title,,,,,,,,,,
The Hateful Eight,Crime Drama Mystery Thriller Western,Quentin_Tarantino,Craig_Stark,2015.000,USA,English,7.900,R,Color,0.765
The Revenant,Adventure Drama Thriller Western,Alejandro_G_Iñárritu,Leonardo_DiCaprio,2015.000,USA,English,8.100,R,Color,0.794
Kill Bill: Vol. 1,Action,Quentin_Tarantino,David_Carradine,2003.000,USA,English,8.100,R,_Black_and_White,0.832
Reservoir Dogs,Crime Drama Thriller,Quentin_Tarantino,Quentin_Tarantino,1992.000,USA,English,8.400,R,Color,0.881
The Great Gatsby,Drama Romance,Baz_Luhrmann,Leonardo_DiCaprio,2013.000,Australia,English,7.300,PG-13,Color,0.911


Similar genre


,genres,plot_keywords,distance
movie_title,,,
Forsaken,Drama Western,,0.198
Unforgiven,Drama Western,englishman leaving_flowers_on_a_grave one_last...,0.198
The Work and the Glory II: American Zion,Drama Western,estrangement missouri mormon mormon_church mor...,0.250
Doc Holliday's Revenge,Western,1880s cowboy low_budget_film shoot_out wild_west,0.335
The Ballad of Gregorio Cortez,Western,,0.350


Similar picture


,director_name,color,title_year,aspect_ratio,duration,country,language,distance
movie_title,,,,,,,,
Inglourious Basterds,Quentin_Tarantino,Color,2009.000,2.350,153.000,USA,English,0.057
The Hateful Eight,Quentin_Tarantino,Color,2015.000,2.760,187.000,USA,English,0.231
Pulp Fiction,Quentin_Tarantino,Color,1994.000,2.350,178.000,USA,English,0.271
Kill Bill: Vol. 2,Quentin_Tarantino,_Black_and_White,2004.000,2.350,137.000,USA,English,0.319
Jackie Brown,Quentin_Tarantino,Color,1997.000,1.850,154.000,USA,English,0.349


Similar cast


,actor_1_name,actor_2_name,actor_3_name,title_year,distance
movie_title,,,,,
Shutter Island,Leonardo_DiCaprio,Joseph_Sikora,Nellie_Sciutto,2010.000,0.467
The Great Gatsby,Leonardo_DiCaprio,Elizabeth_Debicki,Steve_Bisley,2013.000,0.471
Body of Lies,Leonardo_DiCaprio,Simon_McBurney,Michael_Gaston,2008.000,0.474
The Beach,Leonardo_DiCaprio,Virginie_Ledoyen,Peter_Youngblood_Hills,2000.000,0.503
The Quick and the Dead,Leonardo_DiCaprio,Pat_Hingle,Roberts_Blossom,1995.000,0.544


Similar success


,imdb_score,imdb_score,gross,num_user_for_reviews,movie_facebook_likes,num_critic_for_reviews,num_voted_users,distance
movie_title,,,,,,,,
Mad Max: Fury Road,8.100,8.100,153629485.000,1588.000,191000,739.000,552503,0.167
Black Swan,8.000,8.000,106952327.000,1140.000,106000,669.000,551363,0.249
The Wolf of Wall Street,8.200,8.200,116866727.000,1138.000,138000,606.000,780588,0.318
Argo,7.700,7.700,136019448.000,695.000,89000,656.000,452465,0.333
Skyfall,7.800,7.800,304360277.000,1498.000,80000,750.000,522030,0.350


In [11]:
df_2.sort_values('movie_facebook_likes').tail(100).index

Index(['Independence Day: Resurgence', 'The Wolverine',
       'The Secret Life of Walter Mitty', 'Evil Dead', 'Minions', 'Creed',
       'Lincoln', 'Noah', '300: Rise of an Empire', 'Oblivion',
       'Moonrise Kingdom', 'Room', 'Captain America: Civil War',
       'Pride and Prejudice and Zombies', 'Ted', 'The Social Network',
       'The Help', 'Looper', 'Into the Wild', 'Straight Outta Compton',
       'Edge of Tomorrow', 'Warrior', 'Midnight in Paris', 'Idiocracy',
       'Spotlight', 'Suicide Squad', 'Skyfall', 'Warm Bodies', 'Drive', 'Fury',
       'Dark Shadows', 'The Hunger Games: Catching Fire',
       'X-Men: Days of Future Past', 'Terminator Genisys', 'Pacific Rim',
       '12 Years a Slave', 'Lucy', 'The Hobbit: The Desolation of Smaug',
       'Rush', 'Spectre', 'Prisoners', 'Warcraft', 'Maleficent', 'Argo',
       'Into the Woods', 'The Theory of Everything', 'Boyhood',
       'Star Trek Into Darkness', 'Sharknado', 'The Fault in Our Stars',
       'Furious 7', 'Iron Man

##### Prepare export of all our reco

In [18]:
def closest_n(df_pdist, i, n=5, np_already_shown=np.full((movies_count,1), -1)):
    movie_distances = df_pdist[df.loc[i, 'movie_title']]
    np_closest = movie_distances.sort_values()[:n*5]
    # remove movie indexes which have already been suggested(from np_already_shown)
    np_closest = df_movies.loc[np_closest.index, 'movie_id']
    np_closest = np_closest[np.isin(np_closest, np_already_shown[i], invert=True)][:n]
    return (np_closest)

#initiate the list of movies to ignore with each movie's own name
np_already_shown =np.reshape(range(movies_count), (movies_count, 1))
closest_overall = np.array([closest_n(df_pdist_overall,i
                                     ,np_already_shown = np_already_shown
                                     ) for i in range(movies_count)])

np_already_shown = np.hstack((np_already_shown, closest_overall))
closest_genres = np.array([closest_n(df_pdist_genres,i
                                     ,np_already_shown = np_already_shown
                                     ) for i in range(movies_count)])

np_already_shown = np.hstack((np_already_shown, closest_genres))
closest_cast = np.array([closest_n(df_pdist_cast,i
                                     ,np_already_shown = np_already_shown
                                     ) for i in range(movies_count)])


np_already_shown = np.hstack((np_already_shown, closest_cast))
closest_pict = np.array([closest_n(df_pdist_pict,i
                                     ,np_already_shown = np_already_shown
                                     ) for i in range(movies_count)])

np_already_shown = np.hstack((np_already_shown, closest_pict))
closest_success = np.array([closest_n(df_pdist_success,i
                                     ,np_already_shown = np_already_shown
                                     ) for i in range(movies_count)])

### Trying to evaluate results


Calculate percent of chances for movies with same director to end up in our recommendation of either overall or similar pict.

In [63]:
add_percents_reco_same_director =0
nb_movies = 0
for i in range(movies_count):
    nb_common_directors = df[(df['director_name'] == df.loc[i, 'director_name'] ) & (df['director_name'] != '') ].count().max()-1
    if nb_common_directors > 0:
        nb_movies+=1
        df_close_movies =  df.loc[closest_overall[i]].append(df.loc[closest_pict[i]])
        nb_directors_in_reco = df_close_movies[df_close_movies['director_name'] == df.loc[i, 'director_name']].count().max()
        percent_reco_same_director= (nb_directors_in_reco/nb_common_directors)*100
        add_percents_reco_same_director += percent_reco_same_director
        
        
chance_reco_same_director =  add_percents_reco_same_director/nb_movies #77.6

print(chance_reco_same_director, 'chances to find same director movies')

77.67371232899231

Same test with actor_1

In [68]:
add_percents_reco_same_actor =0
nb_movies = 0
for i in range(movies_count):
    nb_common_actors = df[(df['actor_1_name'] == df.loc[i, 'actor_1_name'] ) & (df['actor_1_name'] != '') ].count().max()-1
    if nb_common_actors > 0:
        nb_movies+=1
        df_close_movies =  df.loc[closest_overall[i]].append(df.loc[closest_cast[i]])
        nb_actors_in_reco = df_close_movies[df_close_movies['actor_1_name'] == df.loc[i, 'actor_1_name']].count().max()
        percent_reco_same_actor= (nb_actors_in_reco/nb_common_actors)*100
        add_percents_reco_same_actor += percent_reco_same_actor
        
        
chance_reco_same_actor =  add_percents_reco_same_actor/nb_movies #

print(chance_reco_same_actor, 'chances to find same actor1 movies')

81.31536153012554 chances to find same actor1 movies


In [88]:
print('Average variance of imdb_score in our dataset', df['imdb_score'].var())

Average variance of imdb_score in our dataset 1.271970623049137


In [92]:
sum_variances =0

for i in range(movies_count):
    closect_ids = np.insert(closest_success[i],0,i)
    df_close_movies = df.loc[closect_ids ]
    sum_variances+=df_close_movies['imdb_score'].var()
    
avg_variance = sum_variances/movies_count
print('Average variance of imdb_score in similar success categ:', avg_variance)

Average variance of imdb_score in similar success categ: 0.010970979115812298


In [89]:
sum_variances =0

for i in range(movies_count):
    closect_ids = np.insert(closest_overall[i],0,i)
    df_close_movies = df.loc[closect_ids ]
    sum_variances+=df_close_movies['imdb_score'].var()
    
avg_variance = sum_variances/movies_count
print('Average variance of imdb_score in similar overall:', avg_variance)

Average variance of imdb_score in similar overall: 0.44534384323297943


#### Export

In [21]:
methods_list = ('closest_overall', 'closest_genres', 'closest_cast', 'closest_pict', 'closest_success')
methods={}
for i in methods_list:
    methods[i] = locals()[i]
    

In [22]:
for method in methods:
    np.save(method, methods[method])

In [24]:
for i in methods_list:
    locals()[i] = np.load(i+'.npy')

df_movies[['movie_id', 'movie_title']].to_csv('movies_titles_list.csv')


### Online API URL : http://xmontamat.pythonanywhere.com/
